# Demo for Application Scorecard Model Documentation

This interactive notebook guides model developers through the process of documenting a model with the ValidMind Developer Framework. It uses the [Lending Club](https://www.kaggle.com/code/kmalit/bank-customer-churn-prediction/data) sample dataset from ... to build a simple application scorecard.

As part of the notebook, you will learn how to train a sample model while exploring how the documentation process works:

- Initializing the ValidMind Developer Framework
- Loading a sample dataset provided by the library to train an application scorecard model
- Running a ValidMind test suite to quickly generate documention about the data and model

## ValidMind at a glance

ValidMind's platform enables organizations to identify, document, and manage model risks for all types of models, including AI/ML models, LLMs, and statistical models. As a model developer, you use the ValidMind Developer Framework to automate documentation and validation tests, and then use the ValidMind AI Risk Platform UI to collaborate on model documentation. Together, these products simplify model risk management, facilitate compliance with regulations and institutional standards, and enhance collaboration between yourself and model validators.

If this is your first time trying out ValidMind, you can make use of the following resources alongside this notebook:

- [Get started](https://docs.validmind.ai/guide/get-started.html) — The basics, including key concepts, and how our products work
- [Get started with the ValidMind Developer Framework](https://docs.validmind.ai/guide/get-started-developer-framework.html) — The path for developers, more code samples, and our developer reference


## Before you begin

::: {.callout-tip}

### New to ValidMind?

For access to all features available in this notebook, create a free ValidMind account.

Signing up is FREE — [**Sign up now**](https://app.prod.validmind.ai)
:::

If you encounter errors due to missing modules in your Python environment, install the modules with `pip install`, and then re-run the notebook. For more help, refer to [Installing Python Modules](https://docs.python.org/3/installing/index.html).

## Install the client library

The client library provides Python support for the ValidMind Developer Framework. To install it:


In [1]:
%pip install -q validmind


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Initialize the client library

ValidMind generates a unique _code snippet_ for each registered model to connect with your developer environment. You initialize the client library with this code snippet, which ensures that your documentation and tests are uploaded to the correct model when you run the notebook.

Get your code snippet:

1. In a browser, log into the [Platform UI](https://app.prod.validmind.ai).

2. In the left sidebar, navigate to **Model Inventory** and click **+ Register new model**.

3. Enter the model details and click **Continue**. ([Need more help?](https://docs.validmind.ai/guide/register-models-in-model-inventory.html))

   For example, to register a model for use with this notebook, select:

   - Documentation template: `Binary classification`
   - Use case: `Marketing/Sales - Attrition/Churn Management`

   You can fill in other options according to your preference.

4. Go to **Getting Started** and click **Copy snippet to clipboard**.

Next, replace this placeholder with your own code snippet:

In [2]:
import validmind as vm

vm.init(
  api_host = "https://api.prod.validmind.ai/api/v1/tracking",
  api_key = "9c21ffcabd69730a1c95025af98e79d3",
  api_secret = "7c81ae7002f20acd5192770d0ff4538c7e7206c38a39ddcef1aa862af5be8537",
  project = "clrkpbur5003o19jwmhzbegab"
)

2024-04-10 11:32:25,967 - INFO(validmind.api_client): Connected to ValidMind. Project: Credit Risk Model - Initial Validation (clrkpbur5003o19jwmhzbegab)


## Initialize the Python environment

Next, let's import the necessary libraries and set up your Python environment for data analysis:

In [3]:
import statsmodels.api as sm

%matplotlib inline

### Preview the documentation template

A template predefines sections for your model documentation and provides a general outline to follow, making the documentation process much easier.

You will upload documentation and test results into this template later on. For now, take a look at the structure that the template provides with the `vm.preview_template()` function from the ValidMind library and note the empty sections:

In [4]:
vm.preview_template()

Accordion(children=(Accordion(children=(HTML(value='<p>Empty Section</p>'), HTML(value='<p>Empty Section</p>')…

## Load the sample dataset

The sample dataset used here is provided by the ValidMind library. To be able to use it, you need to import the dataset and load it into a pandas [DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html), a two-dimensional tabular data structure that makes use of rows and columns:

In [5]:
# Import the sample dataset from the library

from validmind.datasets.credit_risk import lending_club

df = lending_club.load_data(source="offline")

df.head()

Loading data from an offline .gz file: /Users/juanvalidmind/Dev/github/validmind/developer-framework/validmind/datasets/credit_risk/datasets/lending_club_loan_data_2007_2014_clean.csv.gz
Attempting to read from .gz file: /Users/juanvalidmind/Dev/github/validmind/developer-framework/validmind/datasets/credit_risk/datasets/lending_club_loan_data_2007_2014_clean.csv.gz
Data loaded successfully.
Rows: 227214, Columns: 23, Missing values: 8859


,loan_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,...,addr_state,dti,inq_last_6mths,open_acc,revol_util,total_acc,total_pymnt,total_pymnt_inv,total_rec_int,last_pymnt_amnt
0,5000,4975.0,36 months,10.65,162.87,B,B2,10+ years,RENT,24000.0,...,AZ,27.65,1.0,3.0,83.7,9.0,5861.071414,5831.78,861.07,171.62
1,2500,2500.0,60 months,15.27,59.83,C,C4,< 1 year,RENT,30000.0,...,GA,1.00,5.0,3.0,9.4,4.0,1008.710000,1008.71,435.17,119.66
2,2400,2400.0,36 months,15.96,84.33,C,C5,10+ years,RENT,12252.0,...,IL,8.72,2.0,2.0,98.5,10.0,3003.653644,3003.65,603.65,649.91
3,10000,10000.0,36 months,13.49,339.31,C,C1,10+ years,RENT,49200.0,...,CA,20.00,1.0,10.0,21.0,37.0,12226.302210,12226.30,2209.33,357.48
4,5000,5000.0,36 months,7.90,156.46,A,A4,3 years,RENT,36000.0,...,AZ,11.20,3.0,9.0,28.3,12.0,5631.377753,5631.38,631.38,161.03


### Prepocess the dataset

Preprocessing performs a number of operations to get ready for the subsequent steps:

- Preprocess the data: Splits the DataFrame (`df`) into multiple datasets (`train_df`, `validation_df`, and `test_df`) using `lending_club.preprocess` to simplify preprocessing.

In [6]:
preprocess_df = lending_club.preprocess(df)
preprocess_df.head()

Filtering 'purpose' to 'debt_consolidation' and 'credit_card':
Rows: 177825
Columns: 23
Missing values: 6590

Filtering out 'grade' F and G:
Rows: 172165
Columns: 23
Missing values: 6427

Filtering out 'sub_grade' F and G:
Rows: 172165
Columns: 23
Missing values: 6427

Filtering out 'home_ownership' OTHER, NONE, ANY:
Rows: 172045
Columns: 23
Missing values: 6425

Dropping specified features:
Rows: 172045
Columns: 13
Missing values: 6350

Dropping rows with any missing values:
Rows: 165695
Columns: 13
Missing values: 0



,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,open_acc,total_acc
0,36,10.65,162.87,B,B2,10,RENT,24000.0,Verified,0,credit_card,3.0,9.0
8,60,14.65,153.45,C,C3,5,OWN,72000.0,Not Verified,0,debt_consolidation,14.0,23.0
9,36,12.69,402.54,B,B5,10,OWN,75000.0,Source Verified,0,debt_consolidation,12.0,34.0
10,36,13.49,305.38,C,C1,0,RENT,30000.0,Source Verified,1,debt_consolidation,4.0,9.0
11,36,9.91,96.68,B,B1,3,RENT,15000.0,Source Verified,0,credit_card,11.0,11.0


### Feature engineering

TBC.

In [7]:
fe_df = lending_club.feature_engineering(preprocess_df)
fe_df.head()

Performing binning with breaks_adj: {'loan_amnt': [5000, 10000, 15000, 20000, 25000], 'int_rate': [10, 15, 20], 'annual_inc': [50000, 100000, 150000]}
[INFO] creating woe binning ...
[INFO] converting into woe values ...
Successfully converted features to WoE values.
Rows: 165695
Columns: 13
Missing values: 0



,loan_status,annual_inc_woe,total_acc_woe,purpose_woe,term_woe,verification_status_woe,open_acc_woe,emp_length_woe,int_rate_woe,installment_woe,home_ownership_woe,sub_grade_woe,grade_woe
0,0,0.272855,0.085175,-0.157770,-0.215312,0.144895,-0.135784,-0.016348,-0.202397,-0.191738,0.114461,-0.336648,-0.396915
8,0,-0.054231,0.032958,0.051351,0.654502,-0.243789,0.023460,-0.016348,-0.202397,-0.191738,0.053303,0.337916,0.186188
9,0,-0.054231,-0.101325,0.051351,-0.215312,0.089350,0.023460,-0.016348,-0.202397,0.128420,0.053303,-0.336648,-0.396915
10,1,0.272855,0.085175,0.051351,-0.215312,0.089350,-0.135784,0.053758,-0.202397,-0.013370,0.114461,-0.336648,0.186188
11,0,0.272855,0.085175,-0.157770,-0.215312,0.089350,0.023460,-0.016348,-1.067745,-0.191738,0.114461,-0.336648,-0.396915


## Train the model

In this section, we focus on constructing and refining our predictive model. Initially, we divide our data, which is based on Weight of Evidence (WoE) features, into training and testing sets (`train_df`, `test_df`). With `lending_club.split`, we employ a simple random split, randomly allocating data points to each set to ensure a mix of examples in both. Additionally, by setting `add_constant=True`, we include an intercept term in our model.



In [8]:
train_df, test_df = lending_club.split(fe_df, add_constant=True)

x_train = train_df.drop(lending_club.target_column, axis=1)
y_train = train_df[lending_club.target_column]
x_test = test_df.drop(lending_club.target_column, axis=1)
y_test = test_df[lending_club.target_column]

# Define the model
model = sm.GLM(
    y_train, 
    x_train, 
    family=sm.families.Binomial())

# Fit the model
model = model.fit()
model.summary()

After splitting the dataset into training and test sets:
Training Dataset:
Rows: 132556
Columns: 14
Missing values: 0

Test Dataset:
Rows: 33139
Columns: 14
Missing values: 0



<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:            loan_status   No. Observations:               132556
Model:                            GLM   Df Residuals:                   132543
Model Family:                Binomial   Df Model:                           12
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -57370.
Date:                Wed, 10 Apr 2024   Deviance:                   1.1474e+05
Time:                        11:32:36   Pearson chi2:                 1.33e+05
No. Iterations:                     5   Pseudo R-squ. (CS):            0.06263
Covariance Type:            nonrobust                                         
===========================================================================================
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                      -1.5464      0.008   -202.301      0.000      -1.561      -1.531
annual_inc_woe              1.1569      0.037     31.143      0.000       1.084       1.230
total_acc_woe               1.0408      0.137      7.592      0.000       0.772       1.309
purpose_woe                 0.2658      0.085      3.128      0.002       0.099       0.432
term_woe                    0.4662      0.022     21.074      0.000       0.423       0.510
verification_status_woe     0.3223      0.047      6.787      0.000       0.229       0.415
open_acc_woe                2.1157      0.171     12.376      0.000       1.781       2.451
emp_length_woe              0.8141      0.202      4.035      0.000       0.419       1.210
int_rate_woe                0.2947      0.030      9.847      0.000       0.236       0.353
installment_woe             0.7049      0.071      9.925      0.000       0.566       0.844
home_ownership_woe          0.6466      0.070      9.189      0.000       0.509       0.784
sub_grade_woe               0.2070      0.041      5.054      0.000       0.127       0.287
grade_woe                   0.3786      0.041      9.344      0.000       0.299       0.458
===========================================================================================
"""

### Compute Scores

In this phase, we translate model predictions into actionable scores using probability estimates generated by our trained model. 

In [9]:
# Compute probabilities from the model
train_probabilities = model.predict(x_train)
test_probabilities = model.predict(x_test)

# Compute scores from the predictions
train_scores = lending_club.compute_scores(train_probabilities)
test_scores = lending_club.compute_scores(test_probabilities)

## Document the model

As part of documenting the model with the ValidMind Developer Framework, you need to preprocess the raw dataset, initialize some training and test datasets, initialize a model object you can use for testing, and then run the full suite of tests.

### Initialize the ValidMind datasets

Before you can run tests, you must first initialize a ValidMind dataset object using the [`init_dataset`](https://docs.validmind.ai/validmind/validmind.html#init_dataset) function from the ValidMind (`vm`) module.

This function takes a number of arguments:

- `dataset` — the dataset that you want to provide as input to tests
- `input_id` - a unique identifier that allows tracking what inputs are used when running each individual test
- `target_column` — a required argument if tests require access to true values. This is the name of the target column in the dataset

With all datasets ready, you can now initialize the raw, processed, training and test datasets (`raw_df`, `clean_df`, `preprocessed_df`, `fe_df`,  `train_df` and `test_df`) created earlier into their own dataset objects using [`vm.init_dataset()`](https://docs.validmind.ai/validmind/validmind.html#init_dataset):

In [10]:
vm_raw_dataset = vm.init_dataset(
    dataset=df,
    input_id="raw_dataset",
    target_column=lending_club.target_column,
)

vm_preprocess_dataset = vm.init_dataset(
    dataset=preprocess_df,
    input_id="preprocess_dataset",
    target_column=lending_club.target_column,
)

vm_fe_dataset = vm.init_dataset(
    dataset=fe_df,
    input_id="fe_dataset",
    target_column=lending_club.target_column,
)

vm_train_ds = vm.init_dataset(
    dataset=train_df,
    input_id="train_dataset",
    target_column=lending_club.target_column,
)

vm_test_ds = vm.init_dataset(
    dataset=test_df, 
    input_id="test_dataset", 
    target_column=lending_club.target_column
)

2024-04-10 11:32:37,151 - INFO(validmind.client): Pandas dataset detected. Initializing VM Dataset instance...
2024-04-10 11:32:38,520 - INFO(validmind.client): Pandas dataset detected. Initializing VM Dataset instance...
2024-04-10 11:32:38,878 - INFO(validmind.client): Pandas dataset detected. Initializing VM Dataset instance...
2024-04-10 11:32:39,137 - INFO(validmind.client): Pandas dataset detected. Initializing VM Dataset instance...
2024-04-10 11:32:39,384 - INFO(validmind.client): Pandas dataset detected. Initializing VM Dataset instance...


TBC.

In [11]:
print(vm_raw_dataset)
print(vm_preprocess_dataset)
print(vm_fe_dataset)
print(vm_train_ds)
print(vm_test_ds)

VMDataset object: 
Input ID: raw_dataset
Target Column: loan_status
Feature Columns: ['loan_amnt', 'funded_amnt_inv', 'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_length', 'home_ownership', 'annual_inc', 'verification_status', 'purpose', 'addr_state', 'dti', 'inq_last_6mths', 'open_acc', 'revol_util', 'total_acc', 'total_pymnt', 'total_pymnt_inv', 'total_rec_int', 'last_pymnt_amnt']
Text Column: None
Extra Columns: {'prediction_columns': {}, 'group_by_column': None}
Type: generic
Target Class Labels: None
Columns: ['loan_amnt', 'funded_amnt_inv', 'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_length', 'home_ownership', 'annual_inc', 'verification_status', 'loan_status', 'purpose', 'addr_state', 'dti', 'inq_last_6mths', 'open_acc', 'revol_util', 'total_acc', 'total_pymnt', 'total_pymnt_inv', 'total_rec_int', 'last_pymnt_amnt']
Index Name: None
Index: [     0      1      2 ... 227211 227212 227213]

VMDataset object: 
Input ID: preprocess_dataset
Target Co

### Initialize a model object

Additionally, you need to initialize a ValidMind model object (`vm_model`) that can be passed to other functions for analysis and tests on the data. You simply intialize this model object with [`vm.init_model()`](https://docs.validmind.ai/validmind/validmind.html#init_model):

In [12]:
vm_model = vm.init_model(
    model,
    input_id="glm_model",
)

### Assign predictions to the datasets

With our model now trained, the next step involves assigning its predictions to our datasets. This is achieved through the use of the `assign_predictions()` method associated with the VM `dataset` object. This method facilitates the linkage of the model's prediction outcomes to the corresponding datasets. It is designed to accept explicitly provided prediction values; however, in their absence, it will automatically compute the predictions.

In [13]:
vm_train_ds.assign_predictions(
    model=vm_model,
    prediction_values=train_probabilities,
)

vm_test_ds.assign_predictions(
    model=vm_model,
    prediction_values=test_probabilities,
)

### Assign scores to the datasets

In the context of credit scorecards, we are interested in computing scores from model predictions, as scorecards fundamentally revolve around these scores rather than raw predictions. To facilitate this process and ensure scores are properly integrated with our datasets, we use the `add_extra_column()` method from the VM `dataset` object. This approach allows us to append scores directly to our data, maintaining a streamlined and coherent dataset ready for analysis.

In [14]:
vm_train_ds.add_extra_column("glm_scores", train_scores)
vm_test_ds.add_extra_column("glm_scores", test_scores)

print(vm_train_ds)
print(vm_test_ds)

2024-04-10 11:32:40,678 - INFO(validmind.vm_models.dataset): Column glm_scores added as an extra column
2024-04-10 11:32:40,684 - INFO(validmind.vm_models.dataset): Column glm_scores added as an extra column


VMDataset object: 
Input ID: train_dataset
Target Column: loan_status
Feature Columns: ['const', 'annual_inc_woe', 'total_acc_woe', 'purpose_woe', 'term_woe', 'verification_status_woe', 'open_acc_woe', 'emp_length_woe', 'int_rate_woe', 'installment_woe', 'home_ownership_woe', 'sub_grade_woe', 'grade_woe']
Text Column: None
Extra Columns: {'prediction_columns': {'glm_model': 'glm_model_prediction'}, 'group_by_column': None, 'glm_scores': 'glm_scores'}
Type: generic
Target Class Labels: None
Columns: ['const', 'loan_status', 'annual_inc_woe', 'total_acc_woe', 'purpose_woe', 'term_woe', 'verification_status_woe', 'open_acc_woe', 'emp_length_woe', 'int_rate_woe', 'installment_woe', 'home_ownership_woe', 'sub_grade_woe', 'grade_woe', 'glm_model_prediction', 'glm_scores']
Index Name: None
Index: [ 75509  72930  59945 ... 183863 203212 170623]

VMDataset object: 
Input ID: test_dataset
Target Column: loan_status
Feature Columns: ['const', 'annual_inc_woe', 'total_acc_woe', 'purpose_woe', 'ter

### Data validation

Data validation is our quality checkpoint before diving into the depths of analysis. Here, we make sure that the data we're working with is accurate, consistent, and ready for rigorous testing. We're looking for anything out of the ordinary that might skew our models or results.

#### Run tests for raw data tests

Initial scrutiny is applied to the unprocessed data to detect any anomalies. This involves:

- *Tabular Description Tables*: Offer a statistical summary for each variable.
- *Missing Values Bar Plot*: Chart out the presence of any missing data.

In [15]:

test= vm.tests.run_test(
    "validmind.data_validation.TabularDescriptionTables",
    inputs = {
        "dataset": vm_raw_dataset
    }
)
test.log()

In [16]:

test= vm.tests.run_test(
    "validmind.data_validation.MissingValuesBarPlot",
    inputs = {
        "dataset": vm_raw_dataset
    }
)
test.log()

#### Run tests for preprocessed data

Prior to modeling, the data undergoes a final review to affirm its preparedness. We conduct:

- *Tabular Description Tables*: Checking structural integrity after preprocessing.
- *IQR Outliers Table*: Searching for statistical anomalies that may impact results.
- *Class Imbalance*: Investigating the proportionality of outcome classes.
- *Tabular Numerical Histograms*: Visualizing distributions for numerical variables.
- *Tabular Categorical Bar Plots*: Analyzing frequencies of categorical variables.
- *Target Rate Bar Plots*: Examining the distribution of the target variable across categories.
- *Pearson Correlation Matrix*: Identifying linear relationships between variables.


In [17]:

test= vm.tests.run_test(
    "validmind.data_validation.TabularDescriptionTables",
    inputs = {
        "dataset": vm_preprocess_dataset
    }
)
test.log()

In [18]:

test= vm.tests.run_test(
    "validmind.data_validation.IQROutliersTable",
    inputs = {
        "dataset": vm_preprocess_dataset
    }
)
test.log()

In [19]:

test= vm.tests.run_test(
    "validmind.data_validation.ClassImbalance",
    inputs = {
        "dataset": vm_preprocess_dataset
    }
)
test.log()

In [20]:

test= vm.tests.run_test(
    "validmind.data_validation.TabularNumericalHistograms",
    inputs = {
        "dataset": vm_preprocess_dataset
    }
)
test.log()

In [21]:

test= vm.tests.run_test(
    "validmind.data_validation.TabularCategoricalBarPlots",
    inputs = {
        "dataset": vm_preprocess_dataset
    }
)
test.log()

In [22]:

test= vm.tests.run_test(
    "validmind.data_validation.TargetRateBarPlots",
    inputs = {
        "dataset": vm_preprocess_dataset
    },
    params = {
        "default_column": lending_club.target_column,
        "columns": None
    }
)
test.log()

The column loan_status is correct and contains only 1 and 0.


In [23]:

test= vm.tests.run_test(
    "validmind.data_validation.PearsonCorrelationMatrix",
    inputs = {
        "dataset": vm_preprocess_dataset
    }
)
test.log()

#### Run tests for WoE analysis

Ensuring data segmentation aligns with predictive value. We look into:

- *WOE Bin Table*: Checking the bins and their corresponding Weight of Evidence values.
- *WOE Bin Plots*: Displaying the distribution of WoE across the data spectrum.

In [24]:

test= vm.tests.run_test(
    "validmind.data_validation.WOEBinTable",
    inputs = {
        "dataset": vm_preprocess_dataset
    },
    params = {
        "breaks_adj": lending_club.breaks_adj
    }
)
test.log()

Running with breaks_adj: {'loan_amnt': [5000, 10000, 15000, 20000, 25000], 'int_rate': [10, 15, 20], 'annual_inc': [50000, 100000, 150000]}
Performing binning with breaks_adj: {'loan_amnt': [5000, 10000, 15000, 20000, 25000], 'int_rate': [10, 15, 20], 'annual_inc': [50000, 100000, 150000]}
[INFO] creating woe binning ...


In [25]:

test= vm.tests.run_test(
    "validmind.data_validation.WOEBinPlots",
    inputs = {
        "dataset": vm_preprocess_dataset
    },
    params = {
        "breaks_adj": lending_club.breaks_adj
    }
)
test.log()

[INFO] creating woe binning ...


### Model validation

This phase verifies the model's predictive power and generalizability. Tests are applied to both training and evaluation stages to ensure robustness and accuracy.

#### Run tests for model training

This step assesses the model's learning process for stability and significance. This involves:

- *Regression Coeffs Plot*: Visual inspection of the model's coefficients.
- *Regression Models Coeffs*: Detailed evaluation of the regression coefficients' values.

In [26]:

test= vm.tests.run_test(
    "validmind.model_validation.statsmodels.RegressionCoeffsPlot",
    inputs = {
        "models": [vm_model]
    }
)
test.log()

In [27]:

test= vm.tests.run_test(
    "validmind.model_validation.statsmodels.RegressionModelsCoeffs",
    inputs = {
        "models": [vm_model]
    }
)
test.log()

#### Run tests for model evaluation

We measure the model's performance on unseen data to validate its predictive power. We conduct:

- *Log Reg Performance Table*: Summarizing logistic regression metrics.
- *GINI Table*: Assessing the discriminatory power of the model.
- *Log Reg Confusion Matrix*: Understanding classification accuracy.
- *Log Reg ROC Curve*: Plotting the trade-off between sensitivity and specificity.
- *Log Reg Prediction Histogram*: Distributing predictions to visualize outcomes.
- *Log Reg Cumulative Prob*: Cumulative probability analysis for predictions.
- *Scorecard Histogram*: Evaluating the distribution of scorecard points.

In [28]:

test = vm.tests.run_test(
    "validmind.model_validation.statsmodels.LogRegPerformanceTable",
    inputs={
        "datasets": [vm_train_ds, vm_test_ds],
        "model": vm_model,
    },
    params = {
        "cut_off_threshold": 0.5
    }
)
test.log()

In [29]:

test= vm.tests.run_test(
    "validmind.model_validation.statsmodels.GINITable",
    inputs = {
        "model": vm_model,
        "datasets": [vm_train_ds, vm_test_ds]
    }
)
test.log()

Computing metrics for Train dataset...
Computing metrics for Test dataset...


In [30]:

test= vm.tests.run_test(
    "validmind.model_validation.statsmodels.LogRegConfusionMatrix",
    inputs = {
        "dataset": vm_test_ds,
        "model": vm_model
    },
    params = {
        "cut_off_threshold": 0.5
    }
)
test.log()

In [31]:

test= vm.tests.run_test(
    "validmind.model_validation.statsmodels.LogRegROCCurve",
    inputs = {
        "model": vm_model,
        "dataset": vm_test_ds
    }
)
test.log()

In [32]:

test= vm.tests.run_test(
    "validmind.model_validation.statsmodels.LogRegPredictionHistogram",
    inputs = {
        "model": vm_model,
        "datasets": [vm_train_ds, vm_test_ds]
    }
)
test.log()

In [33]:

test= vm.tests.run_test(
    "validmind.model_validation.statsmodels.LogRegCumulativeProb",
    inputs = {
        "model": vm_model,
        "datasets": [vm_train_ds, vm_test_ds]
    }
)
test.log()

In [34]:

test= vm.tests.run_test(
    "validmind.model_validation.statsmodels.ScorecardHistogram",
    inputs = {
        "model": vm_model,
        "datasets": [vm_train_ds, vm_test_ds]
    },
    params = {
        "score_column": "glm_scores"
    }
)
test.log()

## Next steps

You can look at the results of this test suite right in the notebook where you ran the code, as you would expect. But there is a better way: view the test results as part of your model documentation right in the ValidMind Platform UI: 

1. In the [Platform UI](https://app.prod.validmind.ai), go to the **Documentation** page for the model you registered earlier.

2. TBC

What you can see now is a more easily consumable version of the tests you just executed, along with other parts of your model documentation that still need to be completed. 

If you want to learn more about where you are in the model documentation process, take a look at [How do I use the framework?](https://docs.validmind.ai/guide/get-started-developer-framework.html#how-do-i-use-the-framework).